# IOS-XE CLEU 2018 Demo

## Connecting to a Device

Let's define some variables:

In [ ]:
# Local CSR 1000v (running under vagrant) -- rtr1
HOST = '127.0.0.1'
PORT = 2223
USER = 'vagrant'
PASS = 'vagrant'

Now let's establish a NETCONF session to that box using ncclient:

In [ ]:
from ncclient import manager
from lxml import etree

def pretty_print(retval):
        print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## Capabilities

Let's look at the capabilities presented by the thing we've just connected to:

In [ ]:
for c in m.server_capabilities:
    print(c)

Ok, that's a bit messy, so let's tidy it up a bit and look, initially, at all the base netconf capabilities:

In [ ]:
nc_caps = [c for c in m.server_capabilities if c.startswith('urn:ietf:params:netconf') and "smiv2" not in c]
for c in nc_caps:
    print(c)

And now let's look at the capabilities that are related to model support:

In [ ]:
import re

for c in m.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))

        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))

        deviations = re.search('deviations=([a-zA-Z0-9\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))

        features = re.search('features=([a-zA-Z0-9\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

## Schema

Let's take a look at playing with schema. First, we can try downloading them, picking one of the modules we got capabilities for.

In [ ]:
# SCHEMA_TO_GET = 'Cisco-IOS-XE-native'
# SCHEMA_TO_GET = 'Cisco-IOS-XE-features'
SCHEMA_TO_GET = 'openconfig-interfaces'

c = m.get_schema(SCHEMA_TO_GET)
print(c.data)

That's not so readable. Let's use a utility called ```pyang``` to get something a bit more readable.

In [ ]:
from subprocess import Popen, PIPE, STDOUT

SCHEMA_TO_GET = 'ietf-interfaces'

c = m.get_schema(SCHEMA_TO_GET)

# Simple pyang tree display
# p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)

# Restrict display depth
# p = Popen(['pyang', '-f', 'tree', '--tree-depth', '2'], stdout=PIPE, stdin=PIPE, stderr=PIPE)

# Restrict display path
p = Popen(['pyang', '-f', 'tree', '--tree-path', 
           'interfaces-state/interface/statistics'], stdout=PIPE, stdin=PIPE, stderr=PIPE)

# Push the data from the get_schema through pyang
stdout_data = p.communicate(input=c.data)[0]

print(stdout_data)

## What About Config?

The ncclient library provides for some simple operations. Let's skip thinking about schemas and stuff like that. Instead let's focus on config and getting end setting it. For that, ncclient provides two methods:

* get_config - takes a target data store and an optional filter
* edit_config - takes a target data store and an XML document with the edit request

### Getting Config

Let's look at some simple requests...

In [ ]:
c = m.get_config(source='running')
pretty_print(c)

Now let's add in a simple filter:

In [ ]:
filter = '''
<native>
 <username/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Retrieve Interface Data (Native Model)

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Retrieve Interface Data (Native Model) With XPath Query

As well as subtree filters, **IOS-XE** supports XPath-based filters.

In [ ]:
filter = '/native/interface/GigabitEthernet/name'

c = m.get_config(source='running', filter=('xpath', filter))
pretty_print(c)

### Retrieve All BGP Data

Now let's look at the BGP native model:

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
 <router>
   <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp"/>
 </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Look At A Specific BGP Neighbor

And can we look at a specific neighbor only? Say the one with id (address) ```192.168.0.1```?

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <router>
    <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
      <id>123</id>
      <neighbor>
        <id>192.168.0.1</id>
      </neighbor>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Create New BGP Neighbor

Ok, so, yes we can get a specific neighbor. Now, can we create a new neighbor? Let's create one with an id of '192.168.1.1', with a remote-as of 666.

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>123</id>
        <neighbor nc:operation="merge">
          <id>192.168.1.1</id>
          <remote-as>666</remote-as>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

Now let's pull back that neighbor:

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <router>
    <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
      <id>123</id>
      <neighbor>
        <id>192.168.1.1</id>
      </neighbor>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

### Modify The BGP Neighbor Description

Can modify something in the neighbor we just created? Let's keep it simple and modify the description:

In [ ]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>123</id>
        <neighbor>
          <id>192.168.1.1</id>
          <description nc:operation="merge">modified description</description>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

### Delete A BGP Neighbor

Might need to do this before creating depending on the state of the router!

In [ ]:
from ncclient.operations import TimeoutExpiredError
from lxml.etree import XMLSyntaxError

edit_data = '''
<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
        <id>123</id>
        <neighbor nc:operation="delete">
          <id>192.168.1.1</id>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except XMLSyntaxError as e:
    print(e)
    print(e.args)
    print(dir(e))
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Other Stuff

Get interface data from native model:

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface>
    <GigabitEthernet>
      <name/>
      <ip/>
    </GigabitEthernet>
  </interface>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

In [ ]:
filter = '''
<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
  <interface>
    <GigabitEthernet>
      <name>1</name>
    </GigabitEthernet>
  </interface>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

## OpenConfig Interface Oper Data Example

Now let's take a look at what we can do when we support the same models on different platforms. In this case we'll take a look at the `openconfig-interfaces` model.

First, let's connect to two different routers, but before running the code below, remember to set up the SSH tunnel...only works inside Cisco network!

```
ssh -N -vvv -L 8300:192.88.84.222:830 einarnn@fe-ucs34.cisco.com
```

You also need to ensure you are running at least IOX-XR 6.3.1 and IOS-XE 16.7.1.

In [1]:
from ncclient import manager
from lxml import etree

# Local CSR 1000v (running under vagrant) -- rtr1
HOST = '127.0.0.1'
PORT = 2223
USER = 'vagrant'
PASS = 'vagrant'

# remote IOS-XR 9000v, accessed over an SSH tunnel
HOST_XR = '127.0.0.1'
PORT_XR = 8300
USER_XR = 'cisco'
PASS_XR = 'cisco'

def pretty_print(retval):
        print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True
    
m_xe = manager.connect(
    host=HOST, port=PORT, username=USER, password=PASS,
    allow_agent=False,
    look_for_keys=False,
    hostkey_verify=False,
    unknown_host_cb=my_unknown_host_cb)

m_xr = manager.connect(
    host=HOST_XR, port=PORT_XR, username=USER_XR, password=PASS_XR,
    allow_agent=False,
    look_for_keys=False,
    hostkey_verify=False,
    unknown_host_cb=my_unknown_host_cb)

Now let's get interface statistics from `openconfig-interfaces` model.

First we define a common filter, with a substitution for the interface name that we can use later:

In [2]:
common_filter = '''
<interfaces xmlns="http://openconfig.net/yang/interfaces">
  <interface>
    <name>%s</name>
    <state>
      <counters/>
    </state>
  </interface>
</interfaces>
'''

In [3]:
#
# IOS-XR Example, get data from the management ethernet
#
c = m_xr.get(filter=('subtree', common_filter % 'MgmtEth0/RP0/CPU0/0'))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="http://openconfig.net/yang/interfaces">
   <interface>
    <name>MgmtEth0/RP0/CPU0/0</name>
    <state>
     <counters>
      <in-multicast-pkts>0</in-multicast-pkts>
      <in-broadcast-pkts>1</in-broadcast-pkts>
      <out-multicast-pkts>0</out-multicast-pkts>
      <out-broadcast-pkts>0</out-broadcast-pkts>
      <out-discards>0</out-discards>
      <in-discards>0</in-discards>
      <in-unknown-protos>0</in-unknown-protos>
      <in-errors>0</in-errors>
      <out-errors>0</out-errors>
      <last-clear>2018-01-25 16:05:32.891 UTC</last-clear>
      <in-octets>155408</in-octets>
      <in-unicast-pkts>1658</in-unicast-pkts>
      <out-octets>1412305</out-octets>
      <out-unicast-pkts>1580</out-unicast-pkts>
     </counters>
    </state>
   </interface>
  </interfaces>
 </data>




In [4]:
#
# IOS-XE Example, get data from Gi1, the commonly used interface for management on CSRs
#
c = m_xe.get(filter=('subtree', common_filter % 'GigabitEthernet1'))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="http://openconfig.net/yang/interfaces">
    <interface>
      <name xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">GigabitEthernet1</name>
      <state>
        <counters>
          <in-octets>391593</in-octets>
          <in-unicast-pkts>3840</in-unicast-pkts>
          <in-broadcast-pkts>0</in-broadcast-pkts>
          <in-multicast-pkts>0</in-multicast-pkts>
          <in-discards>0</in-discards>
          <in-errors>0</in-errors>
          <in-unknown-protos>0</in-unknown-protos>
          <out-octets>3392172</out-octets>
          <out-unicast-pkts>3342</out-unicast-pkts>
          <out-broadcast-pkts>0</out-broadcast-pkts>
          <out-multicast-pkts>0</out-multicast-pkts>
          <out-discards>0</out-discards>
          <out-errors>0</out-errors>
          <last-clear>2018-01-28T21:55:30.000909+00:00</last-clear>
        </counters>
      </state

### Alternate XML Creation

In [ ]:
from lxml import etree

def create_interface_filter(intf_name):
    interfaces = etree.Element(
        "interfaces",
        nsmap = {None: 'http://openconfig.net/yang/interfaces'})
    interface = etree.SubElement(interfaces, "interface")
    etree.SubElement(interface, 'name').text = intf_name
    state = etree.SubElement(interface, 'state')
    etree.SubElement(state, 'counters')
    return interfaces

print(etree.tostring(create_interface_filter('MgmtEth0/RP0/CPU0/0'), pretty_print=True))
print(etree.tostring(create_interface_filter('GigabitEthernet1'), pretty_print=True))

# Enable Debugging

In [ ]:
import logging

handler = logging.StreamHandler()
for l in ['ncclient.transport.ssh', 'ncclient.transport.session', 'ncclient.operations.rpc']:
    logger = logging.getLogger(l)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)